<a href="https://colab.research.google.com/github/Ohmyprogram/Delivery-Robot-TextMining/blob/main/%EA%B8%B0%EC%82%AC%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5%E1%84%89%E1%85%AE%E1%84%8C%E1%85%B5%E1%86%B8_%E1%84%86%E1%85%AE%E1%86%AF%E1%84%85%E1%85%B2%E1%84%85%E1%85%A9%E1%84%87%E1%85%A9%E1%86%BA_20%E1%84%82%E1%85%A7%E1%86%AB(1%E1%84%8B%E1%85%AF%E1%86%AF~6%E1%84%8B%E1%85%AF%E1%86%AF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 기본 설정

### 1-1. Selenium 라이브러리 설치

In [ ]:
# 가상의 브라우저를 컨트롤 할 수 있도록 도와주는 selenium 패키지를 설치합니다.
# 아래 주석을 해지하고 셀을 실행합니다.
# 설치는 한번만 수행하면 되며, 재설치시 Requirement already satisfied: ~ 라는 메시지가 출력됩니다.

!pip install selenium

     ---------------------------------------- 6.3/6.3 MB 9.2 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.2 MB/s eta 0:00:00


### 1-2. 라이브러리 Import 및 Chrome Driver 실행

In [ ]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [ ]:
# Chrome Driver를 호출합니다.
chrome_options = webdriver.ChromeOptions()

# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

# Chrome Driver 파일의 경로를 지정하고 실행합니다.
# Chrome Driver는 아래 링크에서 다운로드 가능합니다.
# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

#service = Service("./chromedriver")    # Windows 운영체제
service = Service("chromedriver.exe")      # MAC, Linux 운영체제
                                       # 경고메시지 출력시 조치 : [시스템 환경설정] > [보안 및 개인정보 보호] > "Chrome Drive ~ 확인없이 허용"

driver = webdriver.Chrome(service=service, options=chrome_options)

## 2. 뉴스기사 및 댓글 수집하기

---

### 2-1. 뉴스기사 정보 입력

In [ ]:
# 수집할 뉴스기사 정보를 입력합니다.
QUERY = "로봇 +물류"          # 필터링 키워드
START_DATE = "2020.01.01" # 필터링 일자 (작성일 기준)
END_DATE = "2020.06.30"
START_PAGE = 1            # 검색결과 저장 페이지 범위 (네이버 뉴스기사는 검색결과 중 최대 4,000페이지만 제공)
END_PAGE = 400
                          # 기사와 댓글을 저장할 파일명
article_filename = "article_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"

### 2-2. 뉴스기사 수집

In [ ]:
f = open(article_filename, "w", encoding="utf-8")
news_count = 0

for page in range(START_PAGE, END_PAGE*10+1, 10):


    URL = "https://search.naver.com/search.naver?&where=news&query=" + QUERY
    URL += "&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds="
    URL += START_DATE + "&de=" + END_DATE + "&docid=&&start=" + str(page) + "&refresh_start=0"
    driver.get(URL)
    time.sleep(1)

    # 검색결과 리스트 불러오기
    try:
        news_list = driver.find_element(By.CLASS_NAME, "list_news").find_elements(By.CLASS_NAME, "bx") # 검색결과 리스트 중 기사 목록
    except:
        break

    # 수집 카운트 업데이트
    news_count += len(news_list)

    # 검색결과 기사 하나씩 탐색
    for news in news_list[:]:
        link_list = news.find_element(By.CLASS_NAME, "info_group").find_elements(By.TAG_NAME, "a")
        if len(link_list) <= 1:
            continue
        # 기사 URL 새탭으로 이동

        article_url = link_list[1].get_attribute("href").strip()
        link_list[1].click()
        time.sleep(1)
        current_window = driver.current_window_handle
        try:
            new_window = [window for window in driver.window_handles if window != current_window][0]
            driver.switch_to.window(new_window)
        except:
            driver.switch_to.window(current_window)
            continue
        time.sleep(1)

        try:
            # 언론사, 작성일자, 본문, 제목을 기사 유형별 방법으로 수집
            if "sports.news" in article_url:
                source_label = driver.find_element(By.ID, "pressLogo").find_element(By.TAG_NAME, "img")
                source = source_label.get_attribute("alt").strip()
                datetime = driver.find_element(By.CLASS_NAME, "info").find_element(By.TAG_NAME, "span").text.replace("기사입력", "").strip()
                content = driver.find_element(By.CLASS_NAME, "news_end").text.strip().replace("\n", " ")
                title = driver.find_element(By.CLASS_NAME, "title").text.strip()
            elif "n.news" in article_url:
                try:
                    try:
                        source_label = driver.find_element(By.ID, "pressLogo").find_element(By.TAG_NAME, "img")
                        source = source_label.get_attribute("alt").strip()
                    except:
                        source_label = driver.find_element(By.CLASS_NAME, "media_end_head_top_logo").find_element(By.TAG_NAME, "img")
                        source = source_label.get_attribute("alt").strip()
                    datetime = driver.find_element(By.CLASS_NAME, "_ARTICLE_DATE_TIME").text.strip()
                    content = driver.find_element(By.CLASS_NAME, "_article_content").text.strip().replace("\n", " ")
                    title = driver.find_element(By.CLASS_NAME, "media_end_head_headline").text.strip()
                except:
                    source_label = driver.find_element(By.CLASS_NAME, "press_logo")
                    source_img = source_label.find_element(By.TAG_NAME, "img")
                    source = source_img.get_attribute("alt").strip()
                    datetime = driver.find_element(By.CLASS_NAME, "author")
                    datetime = datetime.find_element(By.TAG_NAME, "em").text.strip()
                    content = driver.find_element(By.ID, "articeBody").text.strip().replace("\n", " ")
                    title = driver.find_element(By.CLASS_NAME, "end_tit").text.strip()
            # 그 외 기사는 무시
            else:
                driver.close()
                time.sleep(1)
                driver.switch_to.window(current_window)
                continue

            # 댓글(리뷰) 개수
#             review_count_list = driver.find_elements(By.CLASS_NAME, "u_cbox_count")
#             if len(review_count_list) > 0:
#                 review_count = review_count_list[0].text.replace(",", "")
#             else:
#                 review_count = "0"

            # 수집결과 출력
            print(" "*100, end="\r")
            print(str(int(page/10)+1)+"/"+str(END_PAGE)+" Page,", title, end="\r")
            f.write(source + "\t" + datetime + "\t" +
                     article_url + "\t" + title + "\t" + content + "\n")
        except:
            driver.close()
            time.sleep(1)
            driver.switch_to.window(current_window)
            f.flush()
            continue
        driver.close()
        time.sleep(1)
        driver.switch_to.window(current_window)
        f.flush()

f.close()

# 수집종료
print()
print("* 최대", news_count, "개 기사 수집이 완료되었습니다.")
print("* 수집된 기사는 아래 파일에 저장되었습니다.")
print("  :", article_filename)

400/400 Page, [강세 토픽] 면세점 테마, JTC +2.49%, 신세계 +2.42%                                                NA 한줄뉴스>
* 최대 4000 개 기사 수집이 완료되었습니다.
* 수집된 기사는 아래 파일에 저장되었습니다.
  : article_로봇 +물류_2020.01.01_2020.06.30_1.txt
